# Calculate low and high stock solutions concentrations given standard media recipe

This notebook calculates stock concentrations for a media optimization project. Given a standard media recipe, ranges for intervals to be explored, the notebook generates sets of low and high concentrations such that media preparation can be done without dilutions (which reduces the number of operations, hence time, and the number of pipette tips needed). 


Tested using **ART 3.9.4** kernel on jprime.lbl.gov

## Inputs and outputs

#### Required file to run this notebook:
   - `../data/flaviolin/standard_recipe_concentrations.csv`
   
   A file with the standard media recipe, with same units for each component in [mM]. This file also contains a column with **solubility limits** for each component. 
   
   Note that in this study, the target concentration for Kanamycin is given as dilution factor (e.g. 1x).
   
An example of the file content:

| Component | Concentration[mM]   | Solubility[mM]
|------|------|------|
|   MOPS  | 40 | 2389.37 |
| H3BO3 | 0.004 | 700 |
| K2SO4 | 0.29 | 636.98 |


#### Files generated by running this notebook:

   - `stock_concentrations.csv`
   
   - `bounds_file.csv` (optionally, the file with bounds for ART is created)
 
   The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../')

import string
import pandas as pd
import numpy as np
import scipy

from pyDOE import lhs

from core import find_volumes, check_solubility, find_volumes_bulk

### User parameters

In [2]:
user_params = {
    'standard_media_file': '../data/flaviolin/standard_recipe_concentrations.csv',  
    'output_file_path': '../data/flaviolin/', # Folder for output files
    'factor_range': 10,             # How many times higher/lower values from the 
    # standard media you want to explore? If you want to explore different 
    # relative ranges across components, you can specify it below (see cell 6)
    'bounds_file': '../data/flaviolin/Putida_media_bounds.csv', # name of the file with bounds needed for ART
    'well_volume': 1500,            # Total volume of the media content+culture in the well
    'min_volume_transfer': 5,       # Minimal transfer volume of the liquid handler
    'culture_factor': 100,          # Dilution factor for culture, e.g. 100x, 1000x
    } 

In [3]:
culture_volume = user_params['well_volume'] / user_params['culture_factor']


Read the standard media recipe concentrations

In [4]:
df_stand = pd.read_csv(user_params['standard_media_file'])
df_stand = df_stand.set_index("Component")
df_stand

,Concentration,Solubility
Component,,
MOPS[mM],40.00000,2389.37
Tricine[mM],4.00000,500.08
H3BO3[mM],0.00400,700.00
Glucose[mM],20.00000,5045.63
K2SO4[mM],0.29000,636.98
K2HPO4[mM],1.32000,8564.84
FeSO4[mM],0.01000,1645.73
NH4Cl[mM],9.52000,6543.28
MgCl2[mM],0.52000,569.27


Assign exploration ranges for each component. A factor of 1.5 means we want to explore values 50% higher and 50% lower than the values from the standard recipe.

First, the value from `user_params['factor_range']` is assigned to all components. 

In [5]:
num_components = len(df_stand)
df_stand['Factor'] = user_params['factor_range']* np.ones(num_components)

Individual values then can be modified if needed.

In [6]:
df_stand.at['MOPS[mM]', 'Factor'] = 1.0
df_stand.at['Tricine[mM]', 'Factor'] = 1.0
df_stand.at['Glucose[mM]', 'Factor'] = 1.0
df_stand.at['K2HPO4[mM]', 'Factor'] = 5.0
df_stand.at['NH4Cl[mM]', 'Factor'] = 1.5
df_stand.at['Kan', 'Factor'] = 1.

In [7]:
df_stand

,Concentration,Solubility,Factor
Component,,,
MOPS[mM],40.00000,2389.37,1.0
Tricine[mM],4.00000,500.08,1.0
H3BO3[mM],0.00400,700.00,10.0
Glucose[mM],20.00000,5045.63,1.0
K2SO4[mM],0.29000,636.98,10.0
K2HPO4[mM],1.32000,8564.84,5.0
FeSO4[mM],0.01000,1645.73,10.0
NH4Cl[mM],9.52000,6543.28,1.5
MgCl2[mM],0.52000,569.27,10.0


Define target low and high concentration levels:

In [8]:
target_conc_low = df_stand['Concentration'] / df_stand['Factor']
conc_low_round = np.array([round(conc,6) for conc in list(target_conc_low)])
target_conc_low = conc_low_round
target_conc_high = df_stand['Concentration'] * df_stand['Factor']

In [9]:
target_conc_low

array([4.000000e+01, 4.000000e+00, 4.000000e-04, 2.000000e+01,
       2.900000e-02, 2.640000e-01, 1.000000e-03, 6.346667e+00,
       5.200000e-02, 5.000000e+00, 3.000000e-06, 3.000000e-05,
       1.000000e-05, 8.000000e-05, 1.000000e-05, 1.000000e+00])

Save low and high levels of concentrations to a `bounds_file` file needed for ART.

In [10]:
if 'bounds_file' in user_params:
    df_bounds = pd.DataFrame(columns=['Variable', 'Min', 'Max'])
    df_bounds['Variable'] = df_stand.index
    df_bounds['Min'] = target_conc_low
    df_bounds['Max'] = target_conc_high.values
    df_bounds = df_bounds.set_index('Variable')
    df_bounds = df_bounds[df_stand['Factor'] > 1.]
    df_bounds.to_csv(path_or_buf=user_params['bounds_file'])
    display(df_bounds)

,Min,Max
Variable,,
H3BO3[mM],0.000400,0.0400
K2SO4[mM],0.029000,2.9000
K2HPO4[mM],0.264000,6.6000
FeSO4[mM],0.001000,0.1000
NH4Cl[mM],6.346667,14.2800
MgCl2[mM],0.052000,5.2000
NaCl[mM],5.000000,500.0000
(NH4)6Mo7O24[mM],0.000003,0.0003
CoCl2[mM],0.000030,0.0030


## Find a set of low level stock concentrations that can achieve the lowest levels of target concentrations

$$c_s=\frac{c_{t_{\min}} \cdot V_\text{well}}{V_{\min}}$$

In [11]:
min_tip_volume = user_params['min_volume_transfer']
df_low = pd.DataFrame(
    index=df_stand.index,
    columns=["Stock Concentration", "Target Concentration"])
df_low["Target Concentration"] = target_conc_low
df_low["Stock Concentration"] = df_low["Target Concentration"]*user_params['well_volume']/min_tip_volume
df_low

,Stock Concentration,Target Concentration
Component,,
MOPS[mM],12000.0000,40.000000
Tricine[mM],1200.0000,4.000000
H3BO3[mM],0.1200,0.000400
Glucose[mM],6000.0000,20.000000
K2SO4[mM],8.7000,0.029000
K2HPO4[mM],79.2000,0.264000
FeSO4[mM],0.3000,0.001000
NH4Cl[mM],1904.0001,6.346667
MgCl2[mM],15.6000,0.052000


### Check solubility 

Increase the volume transfer, in increments of 5uL, for the components for which concenstrations are not soluble (there is no need to make minimal volume transfers)

$$c^i_{s}=\frac{c^i_{t_{\min}} \cdot V_\text{well}}{V_{\min}+5}$$

In [12]:
if 'Solubility' in df_stand.columns:
    
    nonsol_comp_low = check_solubility(df_low, solubility=df_stand['Solubility'])
    volume_transfer = min_tip_volume

    i = 0
    while len(nonsol_comp_low) > 0:    
        print(f'  Iteration {i}\n')
        volume_transfer += min_tip_volume

        for comp in nonsol_comp_low:
            df_low.at[comp,"Stock Concentration"] = df_low.at[
                comp,"Target Concentration"
            ]*user_params['well_volume']/volume_transfer

        nonsol_comp_low = check_solubility(df_low, solubility=df_stand['Solubility'])
        i += 1
    
else:
    print('Solubility values are not provided and it is assumed the limits are not reached.')
    

Components for which those concentrations are not soluble:
	MOPS[mM]
	Tricine[mM]
	Glucose[mM]
  Iteration 0

Components for which those concentrations are not soluble:
	MOPS[mM]
	Tricine[mM]
  Iteration 1

Components for which those concentrations are not soluble:
	MOPS[mM]
  Iteration 2

Components for which those concentrations are not soluble:
	MOPS[mM]
  Iteration 3

Components for which those concentrations are not soluble:
	MOPS[mM]
  Iteration 4



Check if all volumes are larger than the minimal transfer volume (5 uL)

In [13]:
EPS = 0.000001
volumes, df = find_volumes(
    user_params['well_volume'], 
    components=df_low.index,
    stock_conc_val=df_low['Stock Concentration'].values, 
    target_conc_val=df_low['Target Concentration'].values,
    culture_ratio=user_params['culture_factor']
)
assert (df['Volumes[uL]'].values >= min_tip_volume - EPS).all(), f"Not all volumes are >={min_tip_volume}uL!"

In [14]:
df

,Stock Concentration,Target Concentration,Volumes[uL]
Component,,,
MOPS[mM],2000.0000,40.000000,30.0
Tricine[mM],400.0000,4.000000,15.0
H3BO3[mM],0.1200,0.000400,5.0
Glucose[mM],3000.0000,20.000000,10.0
K2SO4[mM],8.7000,0.029000,5.0
K2HPO4[mM],79.2000,0.264000,5.0
FeSO4[mM],0.3000,0.001000,5.0
NH4Cl[mM],1904.0001,6.346667,5.0
MgCl2[mM],15.6000,0.052000,5.0


Round to 5 digits after decimal point

In [15]:
num_digits = 6
conc = np.array([round(num, num_digits) for num in list(df_low['Stock Concentration'].values)])
df_low['Stock Concentration'] = conc


## Find a set of high level stock concentrations that can achieve the highest levels of target concentrations

Find stock concentrations for the upper limit in the range to explore.

In [16]:
df_high = df_low.copy()
df_high["Target Concentration"] = target_conc_high
df_high["Solubility"] = df_stand['Solubility']

Check if there are feasible volumes for the low level concentrations found above:

In [17]:
try:
    volumes, df = find_volumes(
        user_params['well_volume'],
        components=df_high.index,
        stock_conc_val=df_high['Stock Concentration'].values, 
        target_conc_val=df_high['Target Concentration'].values,
        culture_ratio=user_params['culture_factor']
    )
    feasible_volumes = True
    assert (df['Volumes[uL]'].values >= min_tip_volume - EPS).all(), f"Not all volumes are >={min_tip_volume}uL!"
except AssertionError:
    feasible_volumes = False
    print("No feasible volumes are found!")
    

No feasible volumes are found!


### Find feasible volumes

Increase the current stock concentrations, by 5-fold increments, of components which are the furthest away from the solubility limit  

In [18]:
if not feasible_volumes:
    print("No feasible volumes")
    
    MULTIPL_FACTOR = 5

    success = False
    df = df_high.copy()

    i = 0
    while success is False:
        i += 1

        # Find ratios of solubility over current stock concentrations
        df['Ratio'] = df['Solubility'].values / df['Stock Concentration'].values

        # Find which component is the furthest away from the solubility limit 
        comp = df[df['Ratio'] > MULTIPL_FACTOR]['Ratio'].idxmax()

        # Increase the current stock concentration by a factor
        df.at[comp, 'Stock Concentration'] *= MULTIPL_FACTOR

        # Find if there are feasible volumes for such stock and target concentrations
        try:
            volumes, df_high = find_volumes(
                user_params['well_volume'], 
                components=df.index,
                stock_conc_val=df['Stock Concentration'].values, 
                target_conc_val=df['Target Concentration'].values,
                culture_ratio=user_params['culture_factor']
            )
            success = True
            if success:
                print(f'Iteration {i}:')
                print('Success!')
        except:
            pass
else:
    df_high = df.copy()
    
df_high["Solubility"] = df_stand['Solubility']


No feasible volumes
Iteration 37:
Success!


See what are the calculated volumes

In [19]:
df_high

,Stock Concentration,Target Concentration,Volumes[uL],Solubility
Component,,,,
MOPS[mM],2000.0000,40.0000,30.000000,2389.37
Tricine[mM],400.0000,4.0000,15.000000,500.08
H3BO3[mM],15.0000,0.0400,4.000000,700.00
Glucose[mM],3000.0000,20.0000,10.000000,5045.63
K2SO4[mM],43.5000,2.9000,100.000000,636.98
K2HPO4[mM],396.0000,6.6000,25.000000,8564.84
FeSO4[mM],37.5000,0.1000,4.000000,1645.73
NH4Cl[mM],1904.0001,14.2800,11.249999,6543.28
MgCl2[mM],15.6000,5.2000,500.000000,569.27


### Correct for minimal transfer volumes

If there are volumes that are smaller than the minimum transfer volume, change stock concentrations for those components (decrease the concentrations so that the volume increases).

In [20]:
# Find components with volume transfers smaller than the minimal
comp_small_vol = df_high[
    df_high['Volumes[uL]'] < min_tip_volume - EPS
].index
print(f"{len(comp_small_vol)} component(s) found with volume transfers smaller than the minimal")

# Define new volume transfer to be higher than the minimal, so there is some flexibility
NEW_VOLUME_TRANSFER = 5.0*min_tip_volume

for comp in comp_small_vol:
    factor_diff =  NEW_VOLUME_TRANSFER / (df_high.at[comp, 'Volumes[uL]'])
    print(f'Decreasing the concentration of {comp} by {factor_diff} times')
    df_high.at[comp, 'Stock Concentration'] /= factor_diff
    

7 component(s) found with volume transfers smaller than the minimal
Decreasing the concentration of H3BO3[mM] by 6.25 times
Decreasing the concentration of FeSO4[mM] by 6.249999999999998 times
Decreasing the concentration of (NH4)6Mo7O24[mM] by 781.25 times
Decreasing the concentration of CoCl2[mM] by 781.2500000000001 times
Decreasing the concentration of CuSO4[mM] by 781.25 times
Decreasing the concentration of MnSO4[mM] by 31.25 times
Decreasing the concentration of ZnSO4[mM] by 3906.25 times


Recalculate volumes for corrected stock concentrations:

In [21]:
volumes, df_high_new = find_volumes(
    user_params['well_volume'], 
    components=df_high.index,
    stock_conc_val=df_high['Stock Concentration'].values, 
    target_conc_val=df_high['Target Concentration'].values,
    culture_ratio=user_params['culture_factor']
)
df_high_new

,Stock Concentration,Target Concentration,Volumes[uL]
Component,,,
MOPS[mM],2000.0000,40.0000,30.000000
Tricine[mM],400.0000,4.0000,15.000000
H3BO3[mM],2.4000,0.0400,25.000000
Glucose[mM],3000.0000,20.0000,10.000000
K2SO4[mM],43.5000,2.9000,100.000000
K2HPO4[mM],396.0000,6.6000,25.000000
FeSO4[mM],6.0000,0.1000,25.000000
NH4Cl[mM],1904.0001,14.2800,11.249999
MgCl2[mM],15.6000,5.2000,500.000000


Round to 5 digits after decimal point

In [22]:
df_high = df_high_new.copy()
num_digits = 5
conc = np.array([round(num, num_digits) for num in list(df_high['Stock Concentration'].values)])
df_high['Stock Concentration'] = conc


Create the final dataframe with low and high concentrations and dilution factor for their preparation

In [23]:
df_stock = df_low.copy()
df_stock.rename(columns={'Stock Concentration': 'Low Concentration'}, inplace=True)
df_stock = df_stock.drop(['Target Concentration'], axis='columns')
df_stock['High Concentration'] = df_high['Stock Concentration']
df_stock['Dilution Factor'] = df_stock['High Concentration']/df_stock['Low Concentration']
df_stock

,Low Concentration,High Concentration,Dilution Factor
Component,,,
MOPS[mM],2000.0000,2000.0000,1.0
Tricine[mM],400.0000,400.0000,1.0
H3BO3[mM],0.1200,2.4000,20.0
Glucose[mM],3000.0000,3000.0000,1.0
K2SO4[mM],8.7000,43.5000,5.0
K2HPO4[mM],79.2000,396.0000,5.0
FeSO4[mM],0.3000,6.0000,20.0
NH4Cl[mM],1904.0001,1904.0001,1.0
MgCl2[mM],15.6000,15.6000,1.0


### Test found stock concentrations for different, randomly chosen, target concentrations

Create random target concentrations, sampled using Latin Hypercube, given lower/upper bounds:

In [24]:
n_samples = 100

latin_hc = lhs(
    len(df_stock), samples=n_samples, criterion="maximin"
)

lb = target_conc_low.ravel()
ub = target_conc_high.ravel()

target_conc_val = lb + latin_hc * (ub - lb)

df_target_conc = pd.DataFrame(
    data=target_conc_val, 
    columns=target_conc_high.index
)

Check what are the volumes for random choices of target concentrations within the given ranges:

In [25]:
%%time

df_volumes = find_volumes_bulk(
    df_stock, 
    df_target_conc=df_target_conc, 
    well_volume=user_params['well_volume'],
    min_tip_volume=min_tip_volume,
    culture_ratio=user_params['culture_factor'],
    verbose=0
)

Sucess rate: 100.0%
Sucess rate (water): 100.0%
CPU times: user 1.3 s, sys: 2.35 s, total: 3.65 s
Wall time: 461 ms


In [30]:
df_target_conc

Component,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan
0,40.0,4.0,0.026615,20.0,2.274532,4.748251,0.012808,11.001029,2.237359,233.434659,0.000023,0.000531,0.000760,0.006135,0.000961,1.0
1,40.0,4.0,0.017488,20.0,0.341504,0.937665,0.015323,7.782559,3.435427,382.507039,0.000187,0.002922,0.000140,0.007176,0.000423,1.0
2,40.0,4.0,0.036882,20.0,0.587200,1.874256,0.035522,11.216322,0.575206,297.436913,0.000021,0.002890,0.000790,0.002129,0.000284,1.0
3,40.0,4.0,0.014954,20.0,2.521315,4.107532,0.044526,14.155241,0.080821,229.908164,0.000299,0.002479,0.000698,0.001838,0.000077,1.0
4,40.0,4.0,0.019952,20.0,1.984487,2.094550,0.009382,8.760504,1.455518,467.898898,0.000152,0.001228,0.000058,0.007566,0.000248,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,40.0,4.0,0.025388,20.0,2.784599,2.350203,0.043457,9.738105,1.982932,413.925546,0.000270,0.001500,0.000493,0.003201,0.000663,1.0
96,40.0,4.0,0.012290,20.0,1.855360,4.367184,0.051672,10.091939,4.176978,191.046940,0.000177,0.000710,0.000431,0.002712,0.000301,1.0
97,40.0,4.0,0.005463,20.0,0.166230,4.454021,0.041555,7.102555,4.824471,465.307253,0.000118,0.000312,0.000875,0.002503,0.000106,1.0
98,40.0,4.0,0.022438,20.0,0.099607,3.054049,0.069115,9.369543,3.278418,36.770346,0.000145,0.001866,0.000628,0.004023,0.000616,1.0


### Save the file with stock concentrations

In [26]:
num_digits = 2
dil_fact = np.array([round(num, num_digits) for num in list(df_stock['Dilution Factor'].values)])
df_stock['Dilution Factor'] = dil_fact

Emphasize that kanamycin stock is given in terms of dilution factor:

In [27]:
kan_stock_low = df_stock.at['Kan', 'Low Concentration']
kan_stock_high = df_stock.at['Kan', 'High Concentration']
df_stock.at['Kan'] = [f'{kan_stock_low:.0f}x', f'{kan_stock_high:.0f}x', 1.]
df_stock

,Low Concentration,High Concentration,Dilution Factor
Component,,,
MOPS[mM],2000.0,2000.0,1.0
Tricine[mM],400.0,400.0,1.0
H3BO3[mM],0.12,2.4,20.0
Glucose[mM],3000.0,3000.0,1.0
K2SO4[mM],8.7,43.5,5.0
K2HPO4[mM],79.2,396.0,5.0
FeSO4[mM],0.3,6.0,20.0
NH4Cl[mM],1904.0001,1904.0001,1.0
MgCl2[mM],15.6,15.6,1.0


In [28]:
stock_conc_file = f'{user_params["output_file_path"]}stock_concentrations.csv'

In [29]:
df_stock.to_csv(stock_conc_file)